# Rerank Search Results

This guide demonstrates how to use RedisVL to rerank search results (documents, chunks, or records) based on query relevance. RedisVL supports reranking through Cross-Encoders from [Hugging Face](https://huggingface.co/cross-encoder), [Cohere Rerank API](https://docs.cohere.com/docs/rerank-2), and [VoyageAI Rerank API](https://docs.voyageai.com/docs/reranker).

## Prerequisites

Before you begin, ensure you have:
- Installed RedisVL: `pip install redisvl`
- A running Redis instance ([Redis 8+](https://redis.io/downloads/) or [Redis Cloud](https://redis.io/cloud))

## What You'll Learn

By the end of this guide, you will be able to:
- Rerank search results using HuggingFace Cross-Encoders
- Use the Cohere Rerank API with search results
- Use the VoyageAI Rerank API for result reranking
- Control the number of returned results after reranking

In [1]:
# import necessary modules
import os

## Simple Reranking

Reranking provides a relevance boost to search results generated by
traditional (lexical) or semantic search strategies.

As a simple demonstration, take the passages and user query below:

In [2]:
query = "What is the capital of the United States?"
docs = [
    "Carson City is the capital city of the American state of Nevada. At the 2010 United States Census, Carson City had a population of 55,274.",
    "The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean that are a political division controlled by the United States. Its capital is Saipan.",
    "Charlotte Amalie is the capital and largest city of the United States Virgin Islands. It has about 20,000 people. The city is on the island of Saint Thomas.",
    "Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district. The President of the USA and many major national government offices are in the territory. This makes it the political center of the United States of America.",
    "Capital punishment (the death penalty) has existed in the United States since before the United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states. The federal government (including the United States military) also uses capital punishment."
]

The goal of reranking is to provide a more fine-grained quality improvement to
initial search results. With RedisVL, this would likely be results coming back
from a search operation like full text or vector.

### Using the Cross-Encoder Reranker

To use the cross-encoder reranker we initialize an instance of `HFCrossEncoderReranker` passing a suitable model (if no model is provided, the `cross-encoder/ms-marco-MiniLM-L-6-v2` model is used):  

In [ ]:
from redisvl.utils.rerank import HFCrossEncoderReranker

cross_encoder_reranker = HFCrossEncoderReranker("BAAI/bge-reranker-base")

### Rerank documents with HFCrossEncoderReranker

With the obtained reranker instance we can rerank and truncate the list of
documents based on relevance to the initial query.

In [4]:
results, scores = cross_encoder_reranker.rank(query=query, docs=docs)

In [5]:
for result, score in zip(results, scores):
    print(score, " -- ", result)

0.07461103051900864  --  {'content': 'Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district. The President of the USA and many major national government offices are in the territory. This makes it the political center of the United States of America.'}
0.052202966064214706  --  {'content': 'Charlotte Amalie is the capital and largest city of the United States Virgin Islands. It has about 20,000 people. The city is on the island of Saint Thomas.'}
0.3802356719970703  --  {'content': 'Carson City is the capital city of the American state of Nevada. At the 2010 United States Census, Carson City had a population of 55,274.'}


### Using the Cohere Reranker

To initialize the Cohere reranker you'll need to install the cohere library and provide the right Cohere API Key.

In [32]:
#!pip install cohere

In [33]:
import getpass

# setup the API Key
api_key = os.environ.get("COHERE_API_KEY") or getpass.getpass("Enter your Cohere API key: ")

In [34]:
from redisvl.utils.rerank import CohereReranker

cohere_reranker = CohereReranker(limit=3, api_config={"api_key": api_key})

### Rerank documents with CohereReranker

The following example uses `CohereReranker` to rerank and truncate the list of
documents based on relevance to the initial query.

In [35]:
results, scores = cohere_reranker.rank(query=query, docs=docs)

In [36]:
for result, score in zip(results, scores):
    print(score, " -- ", result)

0.9990564  --  Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district. The President of the USA and many major national government offices are in the territory. This makes it the political center of the United States of America.
0.7516481  --  Capital punishment (the death penalty) has existed in the United States since before the United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states. The federal government (including the United States military) also uses capital punishment.
0.08882029  --  The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean that are a political division controlled by the United States. Its capital is Saipan.


### Working with semi-structured documents

Often times the initial result set includes other metadata and components that could be used to steer the reranking relevancy. To accomplish this, we can set the `rank_by` argument and provide documents with those additional fields.

In [37]:
docs = [
    {
        "source": "wiki",
        "passage": "Carson City is the capital city of the American state of Nevada. At the 2010 United States Census, Carson City had a population of 55,274."
    },
    {
        "source": "encyclopedia",
        "passage": "The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean that are a political division controlled by the United States. Its capital is Saipan."
    },
    {
        "source": "textbook",
        "passage": "Charlotte Amalie is the capital and largest city of the United States Virgin Islands. It has about 20,000 people. The city is on the island of Saint Thomas."
    },
    {
        "source": "textbook",
        "passage": "Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district. The President of the USA and many major national government offices are in the territory. This makes it the political center of the United States of America."
    },
    {
        "source": "wiki",
        "passage": "Capital punishment (the death penalty) has existed in the United States since before the United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states. The federal government (including the United States military) also uses capital punishment."
    }
]

In [38]:
results, scores = cohere_reranker.rank(query=query, docs=docs, rank_by=["passage", "source"])

In [39]:
for result, score in zip(results, scores):
    print(score, " -- ", result)

0.9988121  --  {'source': 'textbook', 'passage': 'Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district. The President of the USA and many major national government offices are in the territory. This makes it the political center of the United States of America.'}
0.5974905  --  {'source': 'wiki', 'passage': 'Capital punishment (the death penalty) has existed in the United States since before the United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states. The federal government (including the United States military) also uses capital punishment.'}
0.059101548  --  {'source': 'encyclopedia', 'passage': 'The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean that are a political division controlled by the United States. Its capital is Saipan.'}


### Using the VoyageAI Reranker

To initialize the VoyageAI reranker you'll need to install the voyaeai library and provide the right VoyageAI API Key.

In [32]:
#!pip install voyageai

In [33]:
import getpass

# setup the API Key
api_key = os.environ.get("VOYAGE_API_KEY") or getpass.getpass("Enter your VoyageAI API key: ")

In [34]:
from redisvl.utils.rerank import VoyageAIReranker

reranker = VoyageAIReranker(model="rerank-lite-1", limit=3, api_config={"api_key": api_key})
# Please check the available models at https://docs.voyageai.com/docs/reranker

### Rerank documents with VoyageAIReranker

The following example uses `VoyageAIReranker` to rerank and truncate the list of
documents based on relevance to the initial query.

In [35]:
results, scores = reranker.rank(query=query, docs=docs)

In [36]:
for result, score in zip(results, scores):
    print(score, " -- ", result)

0.796875  --  Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district. The President of the USA and many major national government offices are in the territory. This makes it the political center of the United States of America.
0.578125  --  Charlotte Amalie is the capital and largest city of the United States Virgin Islands. It has about 20,000 people. The city is on the island of Saint Thomas.
0.5625  --  Carson City is the capital city of the American state of Nevada. At the 2010 United States Census, Carson City had a population of 55,274.


## Next Steps

Now that you understand reranking, explore these related guides:

- [Create Embeddings with Vectorizers](04_vectorizers.ipynb) - Generate embeddings using various providers
- [Query and Filter Data](02_complex_filtering.ipynb) - Build complex filter expressions for search
- [Use Advanced Query Types](11_advanced_queries.ipynb) - Learn about HybridQuery and other query types

## Cleanup

This guide does not create a persistent index, so no cleanup is required.